In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import math
from PIL import Image

In [4]:
# import pretrained models
alexnet = torchvision.models.alexnet(pretrained = True)
resnet = torchvision.models.resnet50(pretrained = True)
vgg = torchvision.models.vgg16(pretrained = True)

# modify the last layer to 9 classes
in_features = alexnet.classifier[-1].in_features
alexnet.classifier[-1] = nn.Linear(in_features, 9)
# alexnet.eval()

in_features = resnet.fc.in_features
resnet.fc = nn.Linear(in_features, 9)
# resnet.eval()

in_features = vgg.classifier[-1].in_features
vgg.classifier[-1] = nn.Linear(in_features, 9)
# vgg.eval()

In [5]:
# load trained parameters

alexnet.load_state_dict(torch.load('alexnet_9l_1_state.pkl'))
resnet.load_state_dict(torch.load('resnet_9l_state_e4.pkl'))
vgg.load_state_dict(torch.load('vgg_9l_state.pkl'))

<All keys matched successfully>

In [6]:
# use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# import data

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_ = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

train_dir = 'train'
data_train =  datasets.ImageFolder(root = train_dir, transform = transform_,
                                       target_transform = None)
val_dir = 'val'
data_val =  datasets.ImageFolder(root = val_dir, transform = transform_,
                                       target_transform = None)

In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(
  data_train,
  batch_size = 10,
  num_workers = 2,
  shuffle = True,
  # pin_memory = torch.cuda.is_available()
)

val_loader = DataLoader(
  data_val,
  batch_size = 10,
  num_workers = 2,
  shuffle = True,
  # pin_memory = torch.cuda.is_available()
)

In [ ]:
# train model
nb_epochs = 100

model = torch.load('alexnet_9l.pkl')
criterion = nn.CrossEntropyLoss()

# model, criterion = alexnet_9l, nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

print("training start")
for e in range(nb_epochs):
  # Freeze the parameters except the last layer
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier[0:-1].parameters():
        p.requires_grad = False

    acc_loss = 0.0
    print('epoch ' + str(e))
    for input, target in iter(train_loader):
        input, target = input.to(device), target.to(device)
        output = model(input)
        loss = criterion(output, target)
        acc_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch loss', loss.item())
    torch.save(model.state_dict(), 'alexnet_9l_1_state.pkl')
    if e%10==0 or e==nb_epochs-1:
        
        torch.save(model, 'alexnet_9l_1.pkl')
    print('epoch loss', acc_loss)

In [ ]:
# train model
nb_epochs = 100

# vgg.load_state_dict(torch.load('vgg_9l_state.pkl'))

# model, criterion = vgg, nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

print("training start for vgg")
for e in range(nb_epochs):
  # Freeze the parameters except the last layer
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier[0:-1].parameters():
        p.requires_grad = False

    acc_loss = 0.0
    print('epoch ' + str(e))
    for input, target in iter(train_loader):
        input, target = input.to(device), target.to(device)
        output = model(input)
        loss = criterion(output, target)
        acc_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch loss', loss.item())
    torch.save(model.state_dict(), 'vgg_9l_state.pkl')
    if e%10==0 or e==nb_epochs-1:
        torch.save(model, 'vgg_9l.pkl')
    print('epoch loss', acc_loss)

In [ ]:
# train model
nb_epochs = 100

# model, criterion = resnet, nn.CrossEntropyLoss()
# model.to(device)
# criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

print("training start for resnet")
for e in range(nb_epochs):
  # Freeze the parameters except the last layer
    for p in model.parameters():
        p.requires_grad = False
    for p in model.fc.parameters():
        p.requires_grad = True

    acc_loss = 0.0
    print('epoch ' + str(e))
    for input, target in iter(train_loader):
        input, target = input.to(device), target.to(device)
        output = model(input)
        loss = criterion(output, target)
        acc_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch loss', loss.item())
    torch.save(model.state_dict(), 'resnet_9l_state_e4.pkl')
    if e%10==0 or e==nb_epochs-1:
        torch.save(model, 'resnet_9l_e4.pkl')
    print('epoch loss', acc_loss)

In [15]:
# test model accuracy on validation set

model = alexnet.to(device)
model.eval()

predictions = torch.tensor([]).to(device)
targets = torch.tensor([]).to(device)

for inputs, target in iter(val_loader):
    inputs, target = inputs.to(device), target.to(device)
    pred = model(inputs).max(dim=1)[1].to(device)
    # acc = (pred == target).float().mean().item()
    # print("accuracy: " + str(acc))
    predictions = torch.cat((predictions, pred), 0)
    targets = torch.cat((targets, target), 0)
    
print('validation accuracy: ', (predictions == targets).float().mean().item())

predictions = torch.tensor([]).to(device)
targets = torch.tensor([]).to(device)

for inputs, target in iter(train_loader):
    inputs, target = inputs.to(device), target.to(device)
    pred = model(inputs).max(dim=1)[1].to(device)
    # acc = (pred == target).float().mean().item()
    # print("accuracy: " + str(acc))
    predictions = torch.cat((predictions, pred), 0)
    targets = torch.cat((targets, target), 0)
    
print('training accuracy: ', (predictions == targets).float().mean().item())

validation accuracy:  0.875268816947937
training accuracy:  0.9866093397140503


In [11]:
# test model accuracy on validation set

# model = alexnet.to(device)
model.eval()

predictions = torch.tensor([]).to(device)
targets = torch.tensor([]).to(device)

for inputs, target in iter(val_loader):
    inputs, target = inputs.to(device), target.to(device)
    pred = model(inputs).max(dim=1)[1].to(device)
    # acc = (pred == target).float().mean().item()
    # print("accuracy: " + str(acc))
    predictions = torch.cat((predictions, pred), 0)
    targets = torch.cat((targets, target), 0)
    
print('validation accuracy for resnet: ', (predictions == targets).float().mean().item())

predictions = torch.tensor([]).to(device)
targets = torch.tensor([]).to(device)

for inputs, target in iter(train_loader):
    inputs, target = inputs.to(device), target.to(device)
    pred = model(inputs).max(dim=1)[1].to(device)
    # acc = (pred == target).float().mean().item()
    # print("accuracy: " + str(acc))
    predictions = torch.cat((predictions, pred), 0)
    targets = torch.cat((targets, target), 0)
    
print('training accuracy for resnet: ', (predictions == targets).float().mean().item())

validation accuracy for resnet:  0.9677419662475586
training accuracy for resnet:  0.9994493722915649


In [10]:
# test model accuracy on validation set

# if hasattr(torch.cuda, 'empty_cache'):
#     torch.cuda.empty_cache()

with torch.no_grad():

    model = vgg.to(device)
    model.eval()

    # torch.backends.cudnn.enabled=False

    predictions = torch.tensor([]).to(device)
    targets = torch.tensor([]).to(device)

    for inputs, target in iter(val_loader):
        inputs, target = inputs.to(device), target.to(device)
        pred = model(inputs).max(dim=1)[1].to(device)
        # acc = (pred == target).float().mean().item()
        # print("accuracy: " + str(acc))
        predictions = torch.cat((predictions, pred), 0)
        targets = torch.cat((targets, target), 0)

    print('validation accuracy for vgg: ', (predictions == targets).float().mean().item())

    predictions = torch.tensor([]).to(device)
    targets = torch.tensor([]).to(device)

    for inputs, target in iter(train_loader):
        inputs, target = inputs.to(device), target.to(device)
        pred = model(inputs).max(dim=1)[1].to(device)
        # acc = (pred == target).float().mean().item()
        # print("accuracy: " + str(acc))
        predictions = torch.cat((predictions, pred), 0)
        targets = torch.cat((targets, target), 0)

    print('training accuracy for vgg: ', (predictions == targets).float().mean().item())

validation accuracy for vgg:  0.9663082957267761
training accuracy for vgg:  0.9955291152000427
